In [1]:
%matplotlib inline

In [2]:
import rasterio as rio
import rasterio.plot
import rasterio.mask
import geopandas as gpds
import fiona
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pds
from pathlib import Path

In [16]:
# Define paths
geo_NI_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\glim_geology\NI_geology_isn_93.shp")
# # Basins A
# combined_wsheds_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\final_watersheds\Basins_A.shp")
# # Define the save paths
# save_path_soils = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\soil_attrs_basins_A.csv")
# save_path_NI_geo = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\NI_geo_attrs_basins_A.csv")

# Basins B
combined_wsheds_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\final_watersheds\Basins_B.shp")
# Define the save paths
save_path_soils = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\soil_attrs_basins_B.csv")
save_path_NI_geo = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\NI_geo_attrs_basins_B.csv")

# ESSD soil properties:

In [17]:
# First we create a variable called nodata
path = path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\european_soil_database_derived_data\ESDD_processed_data_reprojection_in_QGIS\%s_depth_weighted_average.tif" % 'clay_fra') #nodata_set_to_nan
with rio.open(path) as dataset:
    print(dataset.meta)
nodata = dataset.meta['nodata']

# First we process 'clay_fra','grav_fra','soil_tawc','oc_fra', 'silt_fra','soil_poros','root_dep' and 'sand_fra'
wsheds = gpds.read_file(combined_wsheds_path)
wsheds = wsheds.set_index('id')

# Bý til lista með nöfnum sem verða dict keys:
var_list = ['clay_fra','grav_fra','soil_tawc','oc_fra',
            'silt_fra','soil_poros','root_dep','sand_fra']
final_dict = dict()

for var in var_list:
    
    path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\european_soil_database_derived_data\ESDD_processed_data_reprojection_in_QGIS\%s_depth_weighted_average.tif" % var) #nodata_set_to_nan
    src = rio.open(path)

    var_dict = dict()
    for idx in wsheds.index:
        # Clip raster with the watershed shapefile
        out_image, out_transform = rasterio.mask.mask(src, [wsheds.loc[idx]['geometry']], crop=True)
        out_meta = src.meta

        rster = out_image[0]
        rster = rster[rster != nodata] #, np.nan]
        var_dict[idx] = np.nanmean(rster)

        #print('idx %s done' % idx)
    final_dict[var] = var_dict

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 452, 'height': 302, 'count': 1, 'crs': CRS.from_epsg(3057), 'transform': Affine(1327.5220196902653, 0.0, 185128.0905,
       0.0, -1328.6797526490066, 688067.6015)}


In [18]:
# Bedrock depth:
# Now we process the last remaining variable, bedrock depth
# In this case, we are only computing the mean sediment thickness of the land surface that is not covered by lakes or glaciers

var = 'bedrk_dep'

path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\european_soil_database_derived_data\ESDD_processed_data_reprojection_in_QGIS\average_soil_and_sedimentary-deposit_thickness_nodata_is_255_clipped_isn93.tif")

src = rio.open(path)

var_dict = dict()
for idx in wsheds.index:
    # Clip raster with the watershed shapefile
    out_image, out_transform = rasterio.mask.mask(src, [wsheds.loc[idx]['geometry']], crop=True)
    out_meta = src.meta

    rster = out_image[0]
    rster = rster[rster != nodata] #, np.nan]
    rster = rster[rster != 255]
    var_dict[idx] = np.nanmean(rster)

    #print('idx %s done' % idx)
final_dict[var] = var_dict

In [19]:
soil_df = pds.DataFrame(final_dict)
# Convert from % to fraction, and root dep from cm to m
soil_df[['clay_fra','grav_fra','oc_fra','silt_fra','sand_fra','root_dep']]/=100

# Geology

In [20]:
# Load the polygon shapefiles
polygons = gpds.read_file(combined_wsheds_path)
polygons = polygons.set_index('id')
polygons['ID'] = polygons.index
values = gpds.read_file(geo_NI_path)
# Perform an intersection between the two shapefiles
intersection = gpds.overlay(polygons, values, how='intersection')

# Calculate the area of the intersection for each polygon/field combination
intersection['area'] = intersection.geometry.area

# Calculate the total area of each polygon
polygons['total_area'] = polygons.geometry.area

# Calculate the portion of each polygon covered by each field value
portions = intersection.groupby(['ID', 'FLOKKUR'])['area'].sum() / polygons.set_index('ID')['total_area']

# Reshape the portions data into a pivot table with fields as columns and polygons as rows
portions_table = portions.unstack(level='FLOKKUR')

In [21]:
geo_df = portions_table.fillna(0).round(3)
largest_col = portions_table.apply(lambda x: x.idxmax(), axis=1)
df_dominant_class = pds.DataFrame(largest_col)
column_names = ['g%s_fra' % i for i in geo_df.columns]
geo_df.columns = column_names
geo_df['g_dom_NI'] = df_dominant_class

In [22]:
# Save as .csv
soil_df.round(3).to_csv(save_path_soils)
geo_df.round(3).to_csv(save_path_NI_geo)